# 特徴抽出を改良による極性判定の精度向上確認

scikit-learnを用いて自然言語の極性判定を実装します。また、Bag of Wordsにおける特徴の表現方法によって正解率が変化することを確認します。
今回はテストデータは用いず、10分割交差検証による正解率の平均を確認します。


## データ準備

今回用いるデータセット「MovieReview」の取得です。パッケージを用いてデータセットをダウンロードします。

取得できるデータの種類についての説明と提供元のURLが確認できます。


In [1]:
import chazutsu
chazutsu.datasets.MovieReview.polarity().show()

About Moview Review Data
movie review data is annotated by 3 kinds of label (polarity, subjective rating, subjectivity).
see also: http://www.cs.cornell.edu/people/pabo/movie-review-data/


ダウンロードされたデータはローカルに展開されると同時に、
シャッフル・訓練データとテストデータの分割されます。
また、pandas形式で読込を行います。

In [2]:
r = chazutsu.datasets.MovieReview.polarity().download(force=True, test_size=0.0)

Begin downloading the Moview Review Data dataset from http://www.cs.cornell.edu/people/pabo/movie-review-data/review_polarity.tar.gz.
The dataset is saved to /media/brunolw/D/knowledge/nlp_knowledge/src/data/moview_review_data_polarity/review_polarity.tar.gz



Extracting negative data.



Extracting positive data.



Shuffle the extracted dataset.
Done all process! Make below files at /media/brunolw/D/knowledge/nlp_knowledge/src/data/moview_review_data_polarity
 review_polarity.txt


ダウンロードされたデータを先頭から5件確認します。
レビュー内容が「review」、極性が「polarity」として「0(ネガティブ)」、「1(ポジティブ)」として格納されています。


In [3]:
r.data().head()

,polarity,review
0,1,bruce willis is a type-casted actor . in die h...
1,0,"written by david j . schow and john shirley , ..."
2,1,"in tim burton's `sleepy hollow' , there is a m..."
3,0,"after the recent animated debacles of , "" a ru..."
4,0,reading the cast and director for the new mobs...


In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(r.data()['review'], r.data()['polarity'], test_size=0.2, random_state=123)

In [5]:
X_train.head

<bound method NDFrame.head of 1264    as with his other stateside releases , jackie ...
722     deep rising is one of " those " movies . the k...
210     plot : a human space astronaut accidentally fa...
252     senseless ( r ) marlon wayans is a very talent...
297     the heartbreak kid ( reviewed on aug . 26th/19...
                              ...                        
1122    denzel washington is among the many actors thi...
1346    one of the more unusual and suggestively viole...
1406    some critics , including siskel & ebert , are ...
1389    ok , i admit it--i find camp amusement with th...
1534    notting hill's trailer is awful : a laughless ...
Name: review, Length: 1600, dtype: object>

## モデル作成

特徴抽出とモデル選択をパイプラインとして作成します。
今回はCountVectorizerとでBag of Words形式に変換します。（TfidfTransformerでtfidf重み付けも行っています）

単語の数え方における「単語まとまりの単位(1単語 or 2単語)」、「数え方の表現(N回 or 出現有無)」を組み合わせて4パターンのモデルを用意し、精度を比較します。




## 1単語・N回モデル

文書の特徴量として単語まとまりの単位は1単語(unigram)、単語の出現回数をN回としてカウントするBoWを用いるモデル。


In [22]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

def build_pipeline_unigram_multicount():
    text_clf = Pipeline([('vect', CountVectorizer(token_pattern=r'[A-Za-z_]+')),
                      ('tfidf', TfidfTransformer()),
                      ('clf', MultinomialNB()),
    ])
    return text_clf

text_clf_unigram_multicount = build_pipeline_unigram_multicount()

## 1単語・出現有無モデル

文書の特徴量として単語まとまりの単位は1単語(unigram)、単語の出現回数を出現有無としてカウントするBoWを用いるモデル。

In [23]:
def build_pipeline_unigram_binarycount():    
    text_clf = Pipeline([('vect', CountVectorizer(binary=True, token_pattern=r'[A-Za-z_]+'),),
                      ('tfidf', TfidfTransformer()),
                      ('clf', MultinomialNB()),
    ])
    return text_clf

text_clf_unigram_binarycount = build_pipeline_unigram_binarycount()

## 2単語・N回モデル

文書の特徴量として単語まとまりの単位は2単語(unigram)、単語の出現回数をN回としてカウントするBoWを用いるモデル。

In [24]:
def build_pipeline_bigram_multicount():    
    text_clf = Pipeline([('vect', CountVectorizer(ngram_range=(2,2), token_pattern=r'[A-Za-z_]+'),),
                      ('tfidf', TfidfTransformer()),
                      ('clf', MultinomialNB()),
    ])
    return text_clf

text_clf_bigram_multicount = build_pipeline_bigram_multicount()

## 2単語・出現有無モデル

文書の特徴量として単語まとまりの単位は2単語(unigram)、単語の出現回数を出現有無としてカウントするBoWを用いるモデル。


In [25]:
def build_pipeline_bigram_binarycount():    
    text_clf = Pipeline([('vect', CountVectorizer(ngram_range=(2,2), binary=True,token_pattern=r'[A-Za-z_]+'),),
                      ('tfidf', TfidfTransformer()),
                      ('clf', MultinomialNB()),
    ])
    return text_clf

text_clf_bigram_binarycount = build_pipeline_bigram_binarycount()

# Cross Validationによる正解率確認

作成した4つのモデルの正解率を10分割交差検証で確認します。


## 1単語・N回モデル


In [27]:
import pandas as pd
from sklearn.model_selection import cross_val_score

scores = cross_val_score(text_clf_unigram_multicount, X_train, y_train, cv=10)
scores

array([0.80625, 0.85   , 0.75625, 0.7625 , 0.8125 , 0.80625, 0.8375 ,
       0.8375 , 0.80625, 0.79375])

In [28]:
print("Accuracy: %0.3f (+/- %0.3f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.807 (+/- 0.058)


## 1単語・出現有無モデル

In [29]:
scores = cross_val_score(text_clf_unigram_binarycount, X_train, y_train, cv=10)
scores

array([0.81875, 0.86875, 0.775  , 0.775  , 0.83125, 0.81875, 0.825  ,
       0.84375, 0.86875, 0.7875 ])

In [30]:
print("Accuracy: %0.3f (+/- %0.3f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.821 (+/- 0.065)


## 2単語・N回モデル

In [31]:
scores = cross_val_score(text_clf_bigram_multicount, X_train, y_train, cv=10)
scores

array([0.73125, 0.8    , 0.70625, 0.71875, 0.76875, 0.7625 , 0.80625,
       0.825  , 0.80625, 0.7375 ])

In [32]:
print("Accuracy: %0.3f (+/- %0.3f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.766 (+/- 0.079)


## 2単語・出現有無モデル

In [33]:
scores = cross_val_score(text_clf_bigram_binarycount, X_train, y_train, cv=10)
scores

array([0.76875, 0.83125, 0.7375 , 0.73125, 0.8    , 0.7875 , 0.825  ,
       0.8    , 0.8375 , 0.725  ])

In [34]:
print("Accuracy: %0.3f (+/- %0.3f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.784 (+/- 0.080)
